<a href="https://colab.research.google.com/github/Shrason/Projects/blob/main/Webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#import required libraries
import requests
from bs4 import BeautifulSoup as bs


In [23]:
url = 'http://books.toscrape.com/'
response = requests.get(url)

In [24]:
response

<Response [200]>

In [25]:
print(response.text[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
        <meta name="created" content="24th Jun 2016 09:29" />
        <meta name="description" content="" />
        <meta name="viewport" content="width=device-width" />
        <meta name="robots" content="NOARCHIVE,NOCACHE" />

        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
        <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->

        
            <link rel="shortcut icon" href="static/oscar/favicon.

In [26]:
soup = bs(response.text)

In [27]:
soup.find('title').text.strip()

'All products | Books to Scrape - Sandbox'

In [28]:
books_tag = soup.find_all('article', class_ = "product_pod")

In [29]:
book_tag = books_tag[0]

In [30]:
title_tag = book_tag.find('a', title=True)
title_tag['title']

'A Light in the Attic'

In [31]:

rating_tag = book_tag.find('p')['class'][1]
rating_tag

'Three'

In [32]:
price_tag = book_tag.find('p',class_ = 'price_color').text[1:]
price_tag

'£51.77'

In [33]:
link = url + book_tag.find('a')['href']
link

'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

In [34]:
def book_details(book_tag):
  title = book_tag.find('a', title=True)['title']
  rating = book_tag.find('p')['class'][1]
  price  = book_tag.find('p',class_ = 'price_color').text[1:]
  link = link = url + book_tag.find('a')['href']
  return title, rating, price, link

In [35]:
def scrapper(url):
  url = 'http://books.toscrape.com/'
  response = requests.get(url)
  soup = bs(response.text)
  books_tag = soup.find_all('article', class_ = "product_pod")
  books = []
  for book_tag in books_tag:
      books.append(book_details(book_tag))
  return books

In [36]:
books = scrapper(url)
books[:5]

[('A Light in the Attic',
  'Three',
  '£51.77',
  'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'),
 ('Tipping the Velvet',
  'One',
  '£53.74',
  'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html'),
 ('Soumission',
  'One',
  '£50.10',
  'http://books.toscrape.com/catalogue/soumission_998/index.html'),
 ('Sharp Objects',
  'Four',
  '£47.82',
  'http://books.toscrape.com/catalogue/sharp-objects_997/index.html'),
 ('Sapiens: A Brief History of Humankind',
  'Five',
  '£54.23',
  'http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html')]

In [37]:
import pandas as pd
import time
import urllib.parse

In [38]:
def get_soup(url):
    resp = requests.get(url)
    if resp.status_code == 200:
        return bs(resp.text)
    else: return None

def get_all_books(page=3):
    books = []
    for i in range(1, page + 1):
        url = f'http://books.toscrape.com/catalogue/page-{i}.html'
        soup = get_soup(url)
        if soup:
            try:
                book_tags = soup.find_all('article', class_='product_pod')
                for book_tag in book_tags:
                    books.append(book_details(book_tag))
            except:
                print(f'Error reading page {i} . . .')

            time.sleep(5)

    books = pd.DataFrame(books, columns=['title', 'rating', 'price', 'link'])
    return books


In [21]:
df = get_all_books(50)
df.head()

,title,rating,price,link
0,A Light in the Attic,Three,£51.77,http://books.toscrape.com/a-light-in-the-attic...
1,Tipping the Velvet,One,£53.74,http://books.toscrape.com/tipping-the-velvet_9...
2,Soumission,One,£50.10,http://books.toscrape.com/soumission_998/index...
3,Sharp Objects,Four,£47.82,http://books.toscrape.com/sharp-objects_997/in...
4,Sapiens: A Brief History of Humankind,Five,£54.23,http://books.toscrape.com/sapiens-a-brief-hist...


In [39]:
df.shape

(1000, 4)